In [1]:
import pandas as pd

import os
import re


def get_docker_ip():
    docker_host = os.environ.get("DOCKER_HOST", "127.0.0.1")
    docker_ip = re.search("(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})", docker_host)
    return docker_ip.group(1)

# define dummy data for testing
dummy_data = pd.DataFrame(
    [
        {
            "browser": "chrome",
            "sourceMedium": "facebook",
            "users": 12,
            "revenuePerTransaction": 1.0,
            "date": "2019-08-01",
        }
    ]
)

dummy_data["date"] = pd.to_datetime(dummy_data["date"])

dummy_data

,browser,date,revenuePerTransaction,sourceMedium,users
0,chrome,2019-08-01,1.0,facebook,12


## Check that the types are right

In [2]:
dummy_data.dtypes

browser                          object
date                     datetime64[ns]
revenuePerTransaction           float64
sourceMedium                     object
users                             int64
dtype: object

## Get ip of the Docker container (implementation independent)

In [3]:
get_docker_ip()

'192.168.99.100'

## Define connection engine

In [4]:
import sqlalchemy

login_data = {}
login_data["username"] = "test_user"
login_data["password"] = "test_password"
login_data["host"] = get_docker_ip()
login_data["port"] = "3306"
login_data["db_name"] = "test_db"
database_connection = sqlalchemy.create_engine(
    "mysql+pymysql://{username}:{password}@{host}:{port}/{db_name}".format(**login_data)
)

## initially write data to the database

In [5]:
dummy_data.to_sql(
    con=database_connection, name="table_name_for_df", if_exists="replace"
)

## Read back data from the database to verify the data

In [6]:
pd.read_sql_table(table_name='table_name_for_df', con=database_connection)

,index,browser,date,revenuePerTransaction,sourceMedium,users
0,0,chrome,2019-08-01,1.0,facebook,12


## Overwrite existing data

In [7]:
dummy_data2 = dummy_data.copy()
dummy_data2.loc[0, "browser"] = "test"
dummy_data2.to_sql(
    con=database_connection, name="table_name_for_df", if_exists="replace"
)
pd.read_sql_table(table_name="table_name_for_df", con=database_connection)

,index,browser,date,revenuePerTransaction,sourceMedium,users
0,0,test,2019-08-01,1.0,facebook,12


## Appends to existing data

In [8]:
dummy_data3 = dummy_data.copy()
dummy_data3.loc[0, "browser"] = "test2"
dummy_data3.to_sql(
    con=database_connection, name="table_name_for_df", if_exists="append"
)
pd.read_sql_table(table_name="table_name_for_df", con=database_connection)

,index,browser,date,revenuePerTransaction,sourceMedium,users
0,0,test,2019-08-01,1.0,facebook,12
1,0,test2,2019-08-01,1.0,facebook,12
